In [ ]:
import logging
import pandas as pd
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')

    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def get_sectors(conn, cur, id_state):
    query = f"SELECT cod_ibge FROM public.setores_censitarios WHERE CAST(cod_ibge AS TEXT) LIKE '{id_state}%';"
    
    try:
        cur.execute(query)
        result = [row[0] for row in cur.fetchall()]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def get_adults(conn, cur, table, id_sector):
    query = f" \
        SELECT SUM(\"15_19\" + \"20_24\" + \"25_29\" + \"30_34\" + \"35_39\" + \"40_44\" + \"45_49\" + \"50_54\" + \"55_59\" + \"60_64\") \
        FROM public.{table} \
        WHERE id_setor_censitario = {id_sector};"
    
    try:
        cur.execute(query)
        result = cur.fetchone()[0]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def get_total(conn, cur, table, id_sector):
    query = f" \
        SELECT SUM(\"00_04\" + \"05_09\" + \"10_14\" + \"15_19\" + \"20_24\" + \"25_29\" + \"30_34\" + \"35_39\" + \"40_44\" + \"45_49\" + \"50_54\" + \"55_59\" + \"60_64\" + \"65_69\" + \"70_74\" + \"75_79\" + \"80_84\" + \"85_89\" + \"90_94\" + \"95_99\" + \"100\") \
        FROM public.{table} \
        WHERE id_setor_censitario = {id_sector};"
    
    try:
        cur.execute(query)
        result = cur.fetchone()[0]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def insert_data(conn, cur, id_sector, data):
    query = f" \
        INSERT INTO public.populacao_idade_ativa (id_setor_censitario, homens, mulheres, total) \
        VALUES ({id_sector}, {data[0]}, {data[1]}, {data[2]});"
    
    try:
        cur.execute(query)
    except psycopg2.IntegrityError as e:
        conn.rollback()
        logger.error(f"Failed: {e}")
    else:
        conn.commit()

---

In [ ]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [ ]:
conn, cur = open_connection()

In [ ]:
# sectors = get_sectors(conn, cur, 12) # AC - OK
# sectors = get_sectors(conn, cur, 27) # AL
# sectors = get_sectors(conn, cur, 13) # AM
# sectors = get_sectors(conn, cur, 16) # AP
# sectors = get_sectors(conn, cur, 29) # BA
# sectors = get_sectors(conn, cur, 23) # CE
# sectors = get_sectors(conn, cur, 53) # DF
# sectors = get_sectors(conn, cur, 32) # ES
# sectors = get_sectors(conn, cur, 52) # GO
# sectors = get_sectors(conn, cur, 21) # MA
# sectors = get_sectors(conn, cur, 31) # MG
# sectors = get_sectors(conn, cur, 50) # MS
# sectors = get_sectors(conn, cur, 51) # MT
# sectors = get_sectors(conn, cur, 15) # PA
# sectors = get_sectors(conn, cur, 25) # PB
# sectors = get_sectors(conn, cur, 26) # PE
# sectors = get_sectors(conn, cur, 22) # PI
# sectors = get_sectors(conn, cur, 41) # PR
# sectors = get_sectors(conn, cur, 33) # RJ
# sectors = get_sectors(conn, cur, 24) # RN
# sectors = get_sectors(conn, cur, 11) # RO
# sectors = get_sectors(conn, cur, 14) # RR
# sectors = get_sectors(conn, cur, 43) # RS
# sectors = get_sectors(conn, cur, 42) # SC
# sectors = get_sectors(conn, cur, 28) # SE
# sectors = get_sectors(conn, cur, 35) # SP
# sectors = get_sectors(conn, cur, 17) # TO

In [ ]:
logger.info(f"{len(sectors)} found.")

In [ ]:
%%time
for sector in sectors:
    male_adults = get_adults(conn, cur, 'populacao_residente_mas', sector)
    male_total = get_total(conn, cur, 'populacao_residente_mas', sector)
    
    female_adults = get_adults(conn, cur, 'populacao_residente_fem', sector)
    female_total = get_total(conn, cur, 'populacao_residente_fem', sector)
    
    try:
        idx_male = male_adults / male_total
    except Exception as e:
        idx_male = 'NULL'
        
    try:
        idx_female = female_adults / female_total
    except Exception as e:
        idx_female = 'NULL'
        
    try:
        idx_total = (male_adults + female_adults) / (male_total + female_total)
    except Exception as e:
        idx_total = 'NULL'
        
    insert_data(conn, cur, sector, [idx_male, idx_female, idx_total])

In [ ]:
close_connection(conn, cur)